In [1]:
import pickle
import pandas as pd
import numpy as np
import pathlib

In [2]:
data_root = pathlib.Path('.\\TestData\\')
all_test_paths = list(data_root.glob('*.csv'))
all_test_paths = [str(path) for path in all_test_paths]
all_test_paths = sorted(all_test_paths)

## 将时间格式转换成为时间间隔形式

In [4]:
import dateutil
def time_transform(data):
    time_array = data['time'].astype('int64').astype('str').values
    for i in range(len(time_array)):
        time_array[i] = dateutil.parser.parse(time_array[i]).strftime('%Y-%m-%d %H:%M:%S')
    time_Index = pd.to_datetime(time_array)
    time_diff_array = np.zeros((len(time_Index)))
    for i in range(len(time_Index)):
        if i == 0:
            time_diff_array[i] = 10
        else:
            if (time_Index[i] - time_Index[i-1]).seconds >= 600:
                time_diff_array[i] = 0
            else:
                time_diff_array[i] = (time_Index[i] - time_Index[i-1]).seconds
    time_diff_df = pd.DataFrame(time_diff_array, columns=['time_diff'])
    data_final = pd.concat([time_diff_df, data.iloc[:,1:]], axis=1)
    return data_final

In [5]:
for test_path in all_test_paths:
    df = pd.read_csv(test_path)
    data_final = time_transform(df)
    data_final.to_csv(test_path,index=False)